In [ ]:
#|default_exp data.core

# Data Core> Core data structures and utilities for time series data handling

In [ ]:
import tsai_rsimport numpy as npfrom typing import Optional, Tuple

## TSDatasetA lightweight dataset wrapper for time series data.

In [ ]:
class TSDataset:    """Time Series Dataset for classification/regression tasks."""        def __init__(self, X: np.ndarray, y: np.ndarray = None):        self.X = np.asarray(X, dtype=np.float32)        self.y = np.asarray(y) if y is not None else None                if self.X.ndim != 3:            raise ValueError(f"X must be 3D, got {self.X.ndim}D")        def __len__(self) -> int:        return len(self.X)        def __getitem__(self, idx):        if self.y is not None:            return self.X[idx], self.y[idx]        return self.X[idx]        @property    def shape(self) -> Tuple[int, int, int]:        return self.X.shape        @property    def n_channels(self) -> int:        return self.X.shape[1]        @property    def seq_len(self) -> int:        return self.X.shape[2]        @property    def n_classes(self) -> Optional[int]:        if self.y is None:            return None        return len(np.unique(self.y))

In [ ]:
# Test TSDatasetX_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('ECG200')train_ds = TSDataset(X_train, y_train)print(f"Train dataset: {len(train_ds)} samples")print(f"  Shape: {train_ds.shape}")print(f"  Channels: {train_ds.n_channels}")print(f"  Sequence length: {train_ds.seq_len}")print(f"  Classes: {train_ds.n_classes}")

## DataLoaderA simple data loader for batching time series data.

In [ ]:
class TSDataLoader:    """DataLoader for time series data."""        def __init__(self, dataset: TSDataset, batch_size: int = 32,                  shuffle: bool = True, drop_last: bool = False):        self.dataset = dataset        self.batch_size = batch_size        self.shuffle = shuffle        self.drop_last = drop_last        def __len__(self) -> int:        n = len(self.dataset)        if self.drop_last:            return n // self.batch_size        return (n + self.batch_size - 1) // self.batch_size        def __iter__(self):        n = len(self.dataset)        indices = np.arange(n)                if self.shuffle:            np.random.shuffle(indices)                for start in range(0, n, self.batch_size):            end = min(start + self.batch_size, n)            if self.drop_last and end - start < self.batch_size:                break            batch_idx = indices[start:end]                        if self.dataset.y is not None:                yield self.dataset.X[batch_idx], self.dataset.y[batch_idx]            else:                yield self.dataset.X[batch_idx]

In [ ]:
# Test TSDataLoadertrain_loader = TSDataLoader(train_ds, batch_size=16, shuffle=True)print(f"Number of batches: {len(train_loader)}")for i, (xb, yb) in enumerate(train_loader):    print(f"Batch {i}: X shape = {xb.shape}, y shape = {yb.shape}")    if i >= 2:        break

## Data SplitsFunctions for creating train/validation splits.

In [ ]:
def get_splits(n: int, valid_size: float = 0.2,                stratify: np.ndarray = None, seed: int = None):    """Get train/validation split indices."""    if seed is not None:        np.random.seed(seed)        if stratify is not None:        train_idx, valid_idx = [], []        for cls in np.unique(stratify):            cls_idx = np.where(stratify == cls)[0]            np.random.shuffle(cls_idx)            n_valid = int(len(cls_idx) * valid_size)            valid_idx.extend(cls_idx[:n_valid])            train_idx.extend(cls_idx[n_valid:])        return np.array(train_idx), np.array(valid_idx)    else:        indices = np.random.permutation(n)        n_valid = int(n * valid_size)        return indices[n_valid:], indices[:n_valid]

In [ ]:
# Test splitstrain_idx, valid_idx = get_splits(len(X_train), valid_size=0.2, stratify=y_train, seed=42)print(f"Train samples: {len(train_idx)}")print(f"Valid samples: {len(valid_idx)}")